In [ ]:
import os, cv2
import numpy  as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL      import Image
from datetime import datetime

from keras.datasets import cifar10
from keras.models   import Sequential, Model
from keras.layers   import Input, Dense, Reshape, Flatten, Dropout, BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional        import UpSampling2D, Conv2D
from keras.optimizers import Adam

In [ ]:
seed = 0
np.random.seed(seed)
np.random.RandomState(seed)
tf.set_random_seed(seed)

In [ ]:
class DCGAN():
    
    def __init__(self, name="dcgan", shape=(128,128,3), r=5, c=5):
        
        self.path_imgs        = "./images/%s/" % name
        self.path_imgs_latent = "./images/%s/latent/" % name        
        self.path_models      = "./saved_model/%s/"   % name
        for d in ["./images", "./saved_model/", self.path_imgs, self.path_imgs_latent, self.path_models]: 
            if not os.path.exists(d):
                print("Create a new directory, '%s'." % d)
                os.mkdir(d)

        self.shape = shape
        self.z_dim = 100
        self.hist  = []
        
        self.r = r
        self.c = c
        self.check_noise = np.random.uniform(-1, 1, (r * c, self.z_dim))

        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss      = "binary_crossentropy",
                                   optimizer = Adam(lr=0.0002, beta_1=0.5),
                                   metrics   = ["accuracy"])
        self.generator = self.build_generator()

        z   = Input(shape=(self.z_dim,))
        img = self.generator(z)
        self.discriminator.trainable = False
        valid = self.discriminator(img)
        self.combined = Model(z, valid)
        self.combined.compile(loss      = "binary_crossentropy",
                              optimizer = Adam(lr=0.0002, beta_1=0.5))

    def build_generator(self):
        noise_shape = (self.z_dim,)
        model = Sequential()
        model.add(Dense(128 * int(self.shape[0]/4) * int(self.shape[0]/4), activation="relu", input_shape=noise_shape))
        model.add(Reshape((int(self.shape[0]/4), int(self.shape[0]/4), 128)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(self.shape[2], kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        noise = Input(shape=noise_shape)
        img   = model(noise)
        return Model(noise, img)

    def build_discriminator(self):
        img_shape = self.shape
        model = Sequential()
        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        img      = Input(shape=img_shape)
        validity = model(img)
        return Model(img, validity)

    def set_weights(self, g_weights, d_weights):
        # Load generator weights
        if -1 < g_weights.find("/"):
            self.generator.load_weights(g_weights)
        else:
            self.generator.load_weights(self.path_models + g_weights)
        # Load discriminator weights
        if -1 < d_weights.find("/"):
            self.discriminator.load_weights(d_weights)
        else:
            self.discriminator.load_weights(self.path_models + d_weights)

    def train(self, X, iterations, batch_size=128, save_interval=50, model_interval=100):
        start_time = datetime.now()
        half_batch = int(batch_size / 2)
        X_train    = (X.astype(np.float32) - 127.5) / 127.5

        for iteration in range(iterations):
            # Training Discriminator
            idx   = np.random.randint(0, X_train.shape[0], half_batch)
            imgs  = X_train[idx]
            noise = np.random.uniform(-1, 1, (half_batch, self.z_dim))
            gen_imgs = self.generator.predict(noise)
            d_loss_real = self.discriminator.train_on_batch(imgs,     np.ones ((half_batch, 1)))
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            d_loss      = 0.5 * np.add(d_loss_real, d_loss_fake)

            # Training Generator
            noise  = np.random.uniform(-1, 1, (batch_size, self.z_dim))
            g_loss = self.combined.train_on_batch(noise, np.ones((batch_size, 1)))

            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (iteration, d_loss[0], 100 * d_loss[1], g_loss))
            self.hist.append([iteration, d_loss[0], 100 * d_loss[1], g_loss])
            
            if iteration % save_interval == 0:
                self.save_imgs(iteration)
                sta = np.expand_dims(self.check_noise[0], axis=0)
                end = np.expand_dims(self.check_noise[1], axis=0)
                resultImage = self.visualize_interpolation(sta, end)
                cv2.imwrite(self.path_imgs_latent + "latent_%s.png" % iteration, resultImage)
                if iteration % model_interval == 0:
                    self.generator.save    (self.path_models + "generator_%s.h5"     % iteration)
                    self.discriminator.save(self.path_models + "discriminator_%s.h5" % iteration)

    def save_imgs(self, iteration):
        noise    = self.check_noise
        gen_imgs = self.generator.predict(noise)
        # 0-1 rescale
        gen_imgs = 0.5 * gen_imgs + 0.5
        fig, axs = plt.subplots(self.r, self.c)
        cnt = 0
        for i in range(self.r):
            for j in range(self.c):
                if gen_imgs.shape[3]==1:
                    axs[i, j].imshow(gen_imgs[cnt, :, :, 0])  # gray
                else:
                    axs[i, j].imshow(gen_imgs[cnt, :, :, :])  # color      
                axs[i, j].axis("off")
                cnt += 1
        fig.savefig(self.path_imgs + "%s.png" % iteration)
        plt.close()

    def visualize_interpolation(self, start, end, save=True, nbSteps=10):
        steps       = nbSteps
        latentStart = start
        latentEnd   = end
        startImg = self.generator.predict(latentStart)
        endImg   = self.generator.predict(latentEnd)
        vectors  = []
        for alpha in np.linspace(0, 1, steps):
            vector = latentStart * (1 - alpha) + latentEnd * alpha
            vectors.append(vector)
        vectors = np.array(vectors)
        
        resultImage = None
        for i, vec in enumerate(vectors):
            gen_img = np.squeeze(self.generator.predict(vec), axis=0)
            gen_img = (0.5 * gen_img + 0.5) * 255
            interpolatedImage = cv2.cvtColor(gen_img, cv2.COLOR_RGB2BGR)
            interpolatedImage = interpolatedImage.astype(np.uint8)
            resultImage = interpolatedImage if resultImage is None else np.hstack([resultImage, interpolatedImage])
        return resultImage

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
path  = "cifar10"
dcgan = DCGAN(path, shape=(32,32,3))

In [ ]:
dcgan.train(x_train, iterations=1000)